<a href="https://colab.research.google.com/github/ViRoTex/Exercises-Statistics-R-and-Python/blob/main/08_Tablas_pivotaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tablas de pivotaje o dinámicas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb

In [ ]:
titanic = sb.load_dataset("titanic")
titanic.head() 

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
titanic.groupby('sex').mean()

,survived,pclass,age,sibsp,parch,fare,adult_male,alone
sex,,,,,,,,
female,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818,0.000000,0.401274
male,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893,0.930676,0.712305


In [ ]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


In [ ]:
titanic.groupby(['sex', 'pclass'])[['survived']].aggregate('mean').unstack() # sería lo = con pivot_table

survived                    
pclass         1         2         3
sex                                 
female  0.968085  0.921053  0.500000
male    0.368852  0.157407  0.135447

In [ ]:
# pivot_table
titanic.pivot_table('survived', index= 'sex', columns = 'pclass')

pclass,1,2,3
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [ ]:
# para realizar divisiones de variables numéricas se utiliza la función "cut"
# convertimos la variable elegida "age" en factor (columna)
age = pd.cut(titanic['age'], bins=[0,18,25,35,50,65,100], include_lowest = True,  precision = 1, right = False )
age

0      [18.0, 25.0)
1      [35.0, 50.0)
2      [25.0, 35.0)
3      [35.0, 50.0)
4      [35.0, 50.0)
           ...     
886    [25.0, 35.0)
887    [18.0, 25.0)
888             NaN
889    [25.0, 35.0)
890    [25.0, 35.0)
Name: age, Length: 891, dtype: category
Categories (6, interval[int64, left]): [[0, 18) < [18, 25) < [25, 35) < [35, 50) < [50, 65) <
                                        [65, 100)]

In [ ]:
# ahora podemos utilizar la nueva columna creada "age" para hacer una tabla pivotante nueva
# así podemos ver los datos clasificados por los rangos de edad establecidos anteriormente
titanic.pivot_table('survived', index = ['sex', age], columns = 'class')

class                First    Second     Third
sex    age                                    
female [0, 18)    0.875000  1.000000  0.542857
       [18, 25)   1.000000  0.933333  0.500000
       [25, 35)   0.928571  0.923077  0.434783
       [35, 50)   1.000000  0.866667  0.200000
       [50, 65)   0.933333  0.833333  1.000000
male   [0, 18)    1.000000  0.818182  0.232558
       [18, 25)   0.125000  0.050000  0.106667
       [25, 35)   0.550000  0.083333  0.207317
       [35, 50)   0.450000  0.052632  0.069767
       [50, 65)   0.217391  0.090909  0.000000
       [65, 100)  0.166667  0.000000  0.000000

In [ ]:
# para dividir en cuantiles se utiliza "qcut"
fare = pd.qcut(titanic['fare'],q=4)
fare

0       (-0.001, 7.91]
1      (31.0, 512.329]
2       (7.91, 14.454]
3      (31.0, 512.329]
4       (7.91, 14.454]
            ...       
886     (7.91, 14.454]
887     (14.454, 31.0]
888     (14.454, 31.0]
889     (14.454, 31.0]
890     (-0.001, 7.91]
Name: fare, Length: 891, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 7.91] < (7.91, 14.454] < (14.454, 31.0] <
                                           (31.0, 512.329]]

In [ ]:
# los vamos etiquetar
fare = pd.qcut(titanic['fare'],q=4, labels = ["Q1", "Q2", "Q3", "Q4"])
fare

0      Q1
1      Q4
2      Q2
3      Q4
4      Q2
       ..
886    Q2
887    Q3
888    Q3
889    Q3
890    Q1
Name: fare, Length: 891, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [ ]:
# otra tabla a partir de esta nueva creada con los cuantiles
titanic.pivot_table('survived', index=['sex', age], columns = [fare, 'pclass'])

fare               Q1                  Q2                  Q3            \
pclass              1         3         2         3         1         2   
sex    age                                                                
female [0, 18)    NaN  0.800000  1.000000  0.800000       NaN  1.000000   
       [18, 25)   NaN  0.750000  0.800000  0.200000  1.000000  1.000000   
       [25, 35)   NaN  0.200000  1.000000  0.600000  1.000000  0.846154   
       [35, 50)   NaN  0.000000  0.833333  0.000000  1.000000  0.857143   
       [50, 65)   NaN       NaN  0.666667  1.000000  0.500000  1.000000   
male   [0, 18)    NaN  0.000000  0.000000  0.454545       NaN  0.857143   
       [18, 25)   NaN  0.095238  0.071429  0.111111       NaN  0.000000   
       [25, 35)   0.0  0.166667  0.095238  0.222222  0.714286  0.090909   
       [35, 50)   0.0  0.000000  0.111111  0.200000  0.615385  0.000000   
       [50, 65)   NaN  0.000000  0.125000  0.000000  0.200000  0.000000   
       [65, 100)  NaN  0.000000  0.000000       NaN  0.500000       NaN   

fare                              Q4                 
pclass                   3         1    2         3  
sex    age                                           
female [0, 18)    0.461538  0.875000  1.0  0.142857  
       [18, 25)   0.600000  1.000000  1.0  0.000000  
       [25, 35)   0.375000  0.923077  1.0       NaN  
       [35, 50)   0.333333  1.000000  1.0  0.200000  
       [50, 65)        NaN  1.000000  NaN       NaN  
male   [0, 18)    0.285714  1.000000  1.0  0.076923  
       [18, 25)   0.166667  0.125000  0.0       NaN  
       [25, 35)   0.111111  0.500000  0.0  0.750000  
       [35, 50)   0.000000  0.416667  NaN  0.000000  
       [50, 65)        NaN  0.230769  0.0       NaN  
       [65, 100)       NaN  0.000000  NaN       NaN

In [ ]:
# un ejemplo de agregado de una función, aggfunc
titanic.pivot_table(index = "sex", columns = "class", aggfunc = {"survived":sum, "fare":"mean"})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

In [ ]:
titanic.pivot_table('survived', index="sex", columns= "class", margins = True, margins_name ="Total")

class,First,Second,Third,Total
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
Total,0.629630,0.472826,0.242363,0.383838
